# Testando o codigo extraction

In [1]:
#Importando as bibliotecas
import requests
from bs4 import BeautifulSoup
import pandas as pd
from datetime import datetime
import re
import psycopg2
from psycopg2 import sql
from psycopg2.extras import execute_values
from dotenv import load_dotenv
import os
import unicodedata

In [2]:
#Criando os intervalos de tempo para o DataFrame
START_YEAR = 1970
END_YEAR = datetime.now().year - 1

In [3]:
# URLs base para cada tipo de tabela
URL_TEMPLATES = [
    "http://vitibrasil.cnpuv.embrapa.br/index.php?ano={year}&opcao=opt_02",
    "http://vitibrasil.cnpuv.embrapa.br/index.php?ano={year}&opcao=opt_03&subopcao=subopt_01",
    "http://vitibrasil.cnpuv.embrapa.br/index.php?ano={year}&opcao=opt_03&subopcao=subopt_02",
    "http://vitibrasil.cnpuv.embrapa.br/index.php?ano={year}&opcao=opt_03&subopcao=subopt_03",
    "http://vitibrasil.cnpuv.embrapa.br/index.php?ano={year}&opcao=opt_03&subopcao=subopt_04",
    "http://vitibrasil.cnpuv.embrapa.br/index.php?ano={year}&opcao=opt_04",
    "http://vitibrasil.cnpuv.embrapa.br/index.php?ano={year}&opcao=opt_05&subopcao=subopt_01",
    "http://vitibrasil.cnpuv.embrapa.br/index.php?ano={year}&opcao=opt_05&subopcao=subopt_02",
    "http://vitibrasil.cnpuv.embrapa.br/index.php?ano={year}&opcao=opt_05&subopcao=subopt_03",
    "http://vitibrasil.cnpuv.embrapa.br/index.php?ano={year}&opcao=opt_05&subopcao=subopt_04",
    "http://vitibrasil.cnpuv.embrapa.br/index.php?ano={year}&opcao=opt_05&subopcao=subopt_05",
    "http://vitibrasil.cnpuv.embrapa.br/index.php?ano={year}&opcao=opt_06&subopcao=subopt_01",
    "http://vitibrasil.cnpuv.embrapa.br/index.php?ano={year}&opcao=opt_06&subopcao=subopt_02",
    "http://vitibrasil.cnpuv.embrapa.br/index.php?ano={year}&opcao=opt_06&subopcao=subopt_03",
    "http://vitibrasil.cnpuv.embrapa.br/index.php?ano={year}&opcao=opt_06&subopcao=subopt_04"
]

In [4]:
#Verifica a disponibilidade da url
def fetch_page_content(url):
    response = requests.get(url)
    response.raise_for_status()
    return response.content

In [5]:
def standardize_column_name(column):
    column = column.strip()  # Remove extra spaces
    column = column.lower()  # Convert to lowercase
    column = column.replace(' ', '_')  # Replace spaces with underscores
    column = column.replace('(', '')  # Remove parentheses
    column = column.replace(')', '')  # Remove parentheses
    column = column.replace('.', '')  # Remove periods
    column = column.replace('$', 'd')  # Replace $ with D
    
    # Remove accents
    column = unicodedata.normalize('NFKD', column).encode('ascii', 'ignore').decode('utf-8')
    
    return column

In [6]:
def parse_table_content_with_category(content):
    soup = BeautifulSoup(content, 'html.parser')
    table = soup.find('table', class_='tb_base tb_dados')
    headers = [header.text.strip() for header in table.find_all('th')]
    rows = []
    
    if 'Países' not in headers:
        headers.insert(0, 'Categoria')
        current_category = None

        for row in table.find_all('tr'):
            cells = row.find_all('td')

            if len(cells) == 2:
                product = cells[0].text.strip()
                quantity = cells[1].text.strip().replace('.', '').replace('-', '0')
                quantity = int(quantity) if quantity.isdigit() else 0

                # Verifica se o texto está em maiúsculas e não contém números
                if product.isupper() and not any(c.isdigit() for c in product):
                    # Se for uma linha de soma total, assume como a categoria atual
                    current_category = product
                else:
                    # Adiciona a linha ao DataFrame com a categoria atual
                    rows.append([current_category, product, quantity])
    else:
        current_category = None

        for row in table.find_all('tr')[1:]:
            cells = row.find_all('td')
            if len(cells) == 2:
                product = cells[0].text.strip()
                quantity = cells[1].text.strip().replace('.', '').replace('-', '0')
                quantity = int(quantity) if quantity.isdigit() else 0

                # Verifica se o texto está em maiúsculas e não contém números
                if product.isupper() and not any(c.isdigit() for c in product):
                    # Se for uma linha de soma total, assume como a categoria atual
                    current_category = product
                else:
                    # Adiciona a linha ao DataFrame com a categoria atual
                    rows.append([current_category, product, quantity])
            else:
                row_data = [cell.text.strip().replace('.', '').replace('-', '0') for cell in cells]
                rows.append(row_data)

    return headers, rows

In [7]:
#Extrai a tabela para cada ano
def extract_table_data(url, year):
    try:
        content = fetch_page_content(url)
        headers, rows = parse_table_content_with_category(content)
        df = pd.DataFrame(rows, columns=headers)
        df['Ano'] = year
        df.columns = [standardize_column_name(col) for col in df.columns]
        df.fillna(0, inplace=True)
        return df
    except Exception as e:
        print(f"Erro ao extrair dados do ano {year}: {e}")
        return pd.DataFrame()

In [8]:
#Extrai todas as tabelas dentro do intervalo de anos definido
def extract_table_all_data(url_template, start_year, end_year):
    all_data = pd.DataFrame()
    for year in range(start_year, end_year + 1):
        url = url_template.format(year=year)
        year_data = extract_table_data(url, year)
        if not year_data.empty:
            all_data = pd.concat([all_data, year_data], ignore_index=True)
            print(f"Dados do ano {year} extraídos com sucesso.")
    return all_data

In [9]:
#Pega os headers da tabela, para ser usado como coluna no pivot_dataframe
def get_table_headers(url, end_year):
    content = fetch_page_content(url.format(year=end_year))
    headers, _ = parse_table_content_with_category(content)
    return headers

In [10]:
#Pega o titulo da tabela e cria o nome do arquivo CSV com o datetime do dia da extração
def get_filename_from_page(end_year, url):
    content = fetch_page_content(url.format(year=end_year))
    soup = BeautifulSoup(content, 'html.parser')
    p_element = soup.find('p', {'class': 'text_center'})
    if p_element:
        filename_base = p_element.text.strip().replace(f' [{end_year}]', '').replace(' ', '_').replace(',', '')
        timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
        filename = f'{filename_base}_{timestamp}.csv'
        return filename
    return None

In [11]:
#Função para salvar o DataFrame em um CSV
def save_to_csv(df, filename):
    df.to_csv(filename, index=False)

### Sequencia de execução das funções para gerar o DataFrame final

In [ ]:
all_data = extract_table_all_data(URL_TEMPLATES[0], START_YEAR, END_YEAR)
columns = get_table_headers(URL_TEMPLATES[0], END_YEAR)
filename = get_filename_from_page(END_YEAR, URL_TEMPLATES[0])

In [ ]:
columns

In [ ]:
all_data

### Execução para salvar todos os DataFrames extraidos a partir do Web Scraping

In [ ]:
# # Função principal para extrair dados de todos os URLs
# def extract_and_save_all_data(url_templates, start_year, end_year):
#     for url_template in url_templates:
#         all_data = extract_table_all_data(url_template, start_year, end_year)
#         if not all_data.empty:
#             filename = get_filename_from_page(end_year, url_template)
#             if filename:
#                 save_to_csv(all_data, filename)
#                 print(f"Dados salvos em {filename}")
# # Executa a extração de dados
# extract_and_save_all_data(URL_TEMPLATES, START_YEAR, END_YEAR)

In [ ]:
# for url in URL_TEMPLATES:
#     filename = get_filename_from_page(END_YEAR, url)
#     print(filename)

In [ ]:
filename

In [12]:
def process_string(input_string):
    # Remove the file extension
    without_extension = input_string.split('.')[0]
    
    # Remove the date and time part
    parts = without_extension.split('__')
    if len(parts) > 1:
        without_date_time = parts[0]
    else:
        without_date_time = parts[0].rsplit('_', 2)[0]
    
    # Replace special characters and adjust the format
    result = without_date_time.lower().replace('ç', 'c').replace('ã', 'a').replace('é', 'e').replace('í', 'i').replace('ú', 'u').replace('á', 'a').replace('ó', 'o').replace('ê', 'e').replace('ô', 'o').replace(' ', '_').replace('__', '_').replace('_e_', '_').replace('_de_', '_').replace('_', '_')
    
    return result

In [ ]:
filename = process_string(filename)
filename

In [13]:
#Carregando as variaveis de ambiente
load_dotenv()

#Azure
# dbname=os.getenv('PGDATABASE')
# user=os.getenv('PGUSER')
# password=os.getenv('PGPASSWORD')
# host=os.getenv('PGHOST')
# port=os.getenv('PGPORT')

#Heroku
dbname=os.getenv('HDATABASE')
user=os.getenv('HUSER')
password=os.getenv('HPASSWORD')
host=os.getenv('HHOST')
port=os.getenv('HPORT')

In [14]:
# Conectando ao banco de dados PostgreSQL
def conectar_ao_banco(dbname, user, password, host, port):
    conn = psycopg2.connect(
        dbname=dbname,
        user=user,
        password=password,
        host=host,
        port=port
    )
    cur = conn.cursor()
    return conn, cur

In [15]:
# Função para mapear os tipos de dados do Pandas para PostgreSQL
def mapear_tipos_pandas_para_postgresql(dtype):
    if pd.api.types.is_integer_dtype(dtype):
        return "INTEGER"
    elif pd.api.types.is_float_dtype(dtype):
        return "FLOAT"
    elif pd.api.types.is_object_dtype(dtype):
        return "TEXT"
    else:
        return "TEXT"  # Tipo padrão

In [16]:
# Função para criar a tabela no PostgreSQL com base nos dtypes do DataFrame
def criar_tabela_com_dtypes(cur, conn, nome_tabela, df, schema='public'):
    nome_completo_tabela = f"{schema}.{nome_tabela}"
    colunas_com_tipos = ", ".join([f"{col} {mapear_tipos_pandas_para_postgresql(dtype)}" for col, dtype in df.dtypes.items()])
    
    # Criar a query SQL para criar a tabela
    query = f"CREATE TABLE IF NOT EXISTS {nome_completo_tabela} ({colunas_com_tipos});"
    
    # Executar a query
    cur.execute(query)
    conn.commit()

In [17]:
# Inserindo os dados do DataFrame no banco de dados
def inserir_dados(cur, conn, nome_tabela, df):
    for index, row in df.iterrows():
        columns = ', '.join(row.index)
        values = ', '.join(['%s'] * len(row))
        insert_query = sql.SQL(f"INSERT INTO {nome_tabela} ({columns}) VALUES ({values})")
        cur.execute(insert_query, tuple(row))
    conn.commit()

In [18]:
# Função para inserir dados em massa na tabela
def inserir_dados_em_massa(cur, conn, nome_tabela, df):
    """
    Insere dados em massa em uma tabela a partir de um DataFrame utilizando psycopg2.

    Parameters:
    - cur: cursor do banco de dados PostgreSQL.
    - conn: conexão ao banco de dados PostgreSQL.
    - nome_tabela (str): nome da tabela onde os dados serão inseridos.
    - df (DataFrame): pandas DataFrame contendo os dados a serem inseridos.

    Exemplo de uso:
    cur = conn.cursor()
    inserir_dados_em_massa(cur, conn, 'minha_tabela', meu_dataframe)
    """

    # Extraindo as colunas do DataFrame
    columns = list(df.columns)

    # Convertendo os dados do DataFrame em uma lista de tuplas
    data = [tuple(row) for row in df.to_numpy()]

    # Construindo a query de inserção
    query = sql.SQL("INSERT INTO {} ({}) VALUES %s").format(
        sql.Identifier(nome_tabela),
        sql.SQL(', ').join(map(sql.Identifier, columns))
    )
    
    try:
        # Executando a inserção em massa
        execute_values(cur, query, data)
        conn.commit()
        print(f"{cur.rowcount} registros inseridos com sucesso em {nome_tabela}.")
    except Exception as e:
        conn.rollback()
        print(f"Ocorreu um erro ao inserir dados: {e}")

In [19]:
# Fechando a conexão
def fechar_conexao(cur, conn):
    cur.close()
    conn.close()

In [20]:
# Sequência de execução
def executar_sequencia(dbname, user, password, host, port, filename, all_data):
    conn, cur = conectar_ao_banco(dbname, user, password, host, port)
    criar_tabela_com_dtypes(cur, conn, filename, all_data)
    inserir_dados_em_massa(cur, conn, filename, all_data)
    fechar_conexao(cur, conn)

In [21]:
# Função principal para extrair dados de todos os URLs
def extract_and_save_all_data(dbname, user, password, host, port, url_templates, start_year, end_year):
    for url_template in url_templates:
        all_data = extract_table_all_data(url_template, start_year, end_year)
        if not all_data.empty:
            filename = get_filename_from_page(end_year, url_template)
            filename = process_string(filename)
            print(filename)
            executar_sequencia(dbname, user, password, host, port, filename, all_data)


In [22]:
extract_and_save_all_data(dbname, user, password, host, port, URL_TEMPLATES, START_YEAR, END_YEAR)

Dados do ano 1970 extraídos com sucesso.
Dados do ano 1971 extraídos com sucesso.
Dados do ano 1972 extraídos com sucesso.
Dados do ano 1973 extraídos com sucesso.
Dados do ano 1974 extraídos com sucesso.
Dados do ano 1975 extraídos com sucesso.
Dados do ano 1976 extraídos com sucesso.
Dados do ano 1977 extraídos com sucesso.
Dados do ano 1978 extraídos com sucesso.
Dados do ano 1979 extraídos com sucesso.
Dados do ano 1980 extraídos com sucesso.
Dados do ano 1981 extraídos com sucesso.
Dados do ano 1982 extraídos com sucesso.
Dados do ano 1983 extraídos com sucesso.
Dados do ano 1984 extraídos com sucesso.
Dados do ano 1985 extraídos com sucesso.
Dados do ano 1986 extraídos com sucesso.
Dados do ano 1987 extraídos com sucesso.
Dados do ano 1988 extraídos com sucesso.
Dados do ano 1989 extraídos com sucesso.
Dados do ano 1990 extraídos com sucesso.
Dados do ano 1991 extraídos com sucesso.
Dados do ano 1992 extraídos com sucesso.
Dados do ano 1993 extraídos com sucesso.
Dados do ano 199

C:\Users\ernan\AppData\Local\Temp\ipykernel_34104\1104401823.py:9: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df.fillna(0, inplace=True)
C:\Users\ernan\AppData\Local\Temp\ipykernel_34104\1104401823.py:9: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df.fillna(0, inplace=True)


Dados do ano 1970 extraídos com sucesso.
Dados do ano 1971 extraídos com sucesso.


C:\Users\ernan\AppData\Local\Temp\ipykernel_34104\1104401823.py:9: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df.fillna(0, inplace=True)


Dados do ano 1972 extraídos com sucesso.


...

Dados do ano 2021 extraídos com sucesso.


C:\Users\ernan\AppData\Local\Temp\ipykernel_34104\1104401823.py:9: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df.fillna(0, inplace=True)


Dados do ano 2022 extraídos com sucesso.


C:\Users\ernan\AppData\Local\Temp\ipykernel_34104\1104401823.py:9: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df.fillna(0, inplace=True)


Dados do ano 2023 extraídos com sucesso.
uvas_sem_classificacao_processadas
8 registros inseridos com sucesso em uvas_sem_classificacao_processadas.
Dados do ano 1970 extraídos com sucesso.
Dados do ano 1971 extraídos com sucesso.
Dados do ano 1972 extraídos com sucesso.
Dados do ano 1973 extraídos com sucesso.
Dados do ano 1974 extraídos com sucesso.
Dados do ano 1975 extraídos com sucesso.
Dados do ano 1976 extraídos com sucesso.
Dados do ano 1977 extraídos com sucesso.
Dados do ano 1978 extraídos com sucesso.
Dados do ano 1979 extraídos com sucesso.
Dados do ano 1980 extraídos com sucesso.
Dados do ano 1981 extraídos com sucesso.
Dados do ano 1982 extraídos com sucesso.
Dados do ano 1983 extraídos com sucesso.
Dados do ano 1984 extraídos com sucesso.
Dados do ano 1985 extraídos com sucesso.
Dados do ano 1986 extraídos com sucesso.
Dados do ano 1987 extraídos com sucesso.
Dados do ano 1988 extraídos com sucesso.
Dados do ano 1989 extraídos com sucesso.
Dados do ano 1990 extraídos com